# Tabular Playground Series - Aug 2021

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
data_test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')
data_submit = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
data_train.head()

In [ ]:
data_train.isnull().any()

In [ ]:
graph_train=[]
for i in range(100):
    w="f"+str(i)
    graph_train.append(w)
print(graph_train)

In [ ]:
for i in range(2):
    x=int(i)
    plt.hist(graph_train[x])
    plt.title(i)
    plt.show()

In [ ]:
data_train.shape

In [ ]:
data_train.shape

___
# Split the Data

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data_train.drop('loss',axis=1), 
                                                    data_train['loss'], test_size=0.2, 
                                                    random_state=0)

In [ ]:
X = data_train.drop(['id','loss'],axis=1)
y = data_train['loss']
X.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
standard = StandardScaler()
X_train = standard.fit_transform(X)
X_test =  standard.transform(data_test.drop('id',axis=1))

___
## Training the Data

In [ ]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

In [ ]:
lr = LinearRegression()
xgb = XGBRegressor(n_estimators = 100,random_state = 0)
dt = DecisionTreeRegressor()
gb = GradientBoostingRegressor(random_state = 0)
cat = CatBoostRegressor()
lg = LGBMRegressor()

In [ ]:
import time
def model_result(model):
    start = time.time()
    model.fit(X_train,y)
    mean_result = cross_val_score(model,X_train,y,cv=5,scoring='neg_mean_squared_error')
    stop = time.time()
    print(f"Time spend : {round((stop-start),2)}s")
    return -1*mean_result.mean()

In [ ]:
print("Linear Regression :  {}".format(np.sqrt(model_result(lr))))

In [ ]:
print("Decision Tree : {}".format(np.sqrt(model_result(dt))))

In [ ]:
print("XGBoost  : {}".format(np.sqrt(model_result(xgb))))

In [ ]:
print("CatBoost  : {}".format(np.sqrt(model_result(cat))))

In [ ]:
print("Light : {}".format(np.sqrt(model_result(lg))))

___
## Preparing the Submission and Export


In [ ]:
prediction=cat.predict(X_test)

output = pd.DataFrame({'id': data_test.id,
                       'loss': prediction})
output.to_csv('submission.csv', index=False)